Exercise: Rolling Update
-----------------

In this exercise, we'll create multiple pods from the same image, each with a ReplicaSet, Deployment, Service, and **Ingress**.

This happens in a separate namespace to be able to display the results in a targeted manner:

In [ ]:
! kubectl create namespace depl

We create the Pod, ReplicaSet, Deployment and Service.

Specifying `replica: 5` is sufficient for the `Replica-Set`.

The pods follow from `spec.containers`.

Multiple resources can be combined into one file using `---`. Deployment and service here.

In [ ]:
%%bash
cat <<%EOF% | kubectl --namespace depl apply -f -
apiVersion: v1
kind: Service
metadata:
  name: bpmn-frontend
  labels:
    app: bpmn-frontend
    group: web
    tier: frontend
spec:
  ports:
  - port: 80
    targetPort: 80  
    protocol: TCP
  selector:
    app: bpmn-frontend
  type: LoadBalancer          
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: bpmn-frontend
spec:
  replicas: 5
  selector:
    matchLabels:
      app: bpmn-frontend
  template:
    metadata:
      labels:
        app: bpmn-frontend
        group: web
        tier: frontend
    spec:
      containers:
      - name: bpmn-frontend
        image: registry.gitlab.com/mc-b/misegr/bpmn-frontend:V0.2
        imagePullPolicy: IfNotPresent        
        ports:
        - containerPort: 80
          name: bpmn-frontend
%EOF%

To check, we output the generated resources:

In [ ]:
# ! kubectl get pod,deployment,replicaset,service --namespace depl -o wide
! kube-lineage --namespace depl deployment bpmn-frontend -d 2 --exclude-types events 

First, let's look at the output of the running pod, this time via the URL of the Kubernetes API server and the prefix attached via Ingress resources.

The prefix `frontend` is permanently stored in the ingress resource, `index.html` results from how the container is structured (Apache server with an HTML file).

In [ ]:
! echo "BPMN Frontend: http://$(cat ~/work/server-ip):$(kubectl get service --namespace depl bpmn-frontend -o=jsonpath='{ .spec.ports[0].nodePort }')/frontend/index.html"

However, we don't want the latest (`V0.2`) version of `bpmn-frontend` but version `V1.0`, so we are doing a rolling update, or changing the version number behind the image name.

In [ ]:
! kubectl set image deployment/bpmn-frontend bpmn-frontend=registry.gitlab.com/mc-b/misegr/bpmn-frontend:V1.0 --namespace depl

We can display the changes:

In [ ]:
#! kubectl get pod,deployment,replicaset,service --namespace depl -o wide   # Unuebersichtlich!
#! kubectl describe deployment/bpmn-frontend --namespace depl
! kube-lineage --namespace depl deployment bpmn-frontend -d 2  

In [ ]:
! echo "BPMN Frontend: http://$(cat ~/work/server-ip):$(kubectl get service --namespace depl bpmn-frontend -o=jsonpath='{ .spec.ports[0].nodePort }')/frontend/index.html"

The version number `V1.0` should now be displayed in the title.

- - -

RollOut
-----------

If the new version of the software doesn't work as expected, we can revert to the previous version.

This is possible because Kubernetes stores deployment rollout history in the form of revisions.


In [ ]:
! kubectl rollout history deployment/bpmn-frontend --namespace depl

Two revisions should appear. The current one is revision 2.

To revert to the previous revision, we use:

In [ ]:
! kubectl rollout undo deployment/bpmn-frontend --namespace depl

In [ ]:
! kubectl rollout history deployment/bpmn-frontend --namespace depl

Clean up

In [ ]:
! kubectl delete namespace depl

***
Questions
======

Answer the questions individually or in groups and compare them with the answers

---

What is the purpose of a ReplicaSet?
<details><summary>Answer</summary><p>
Ensures N pods are running, too few pods are started, too many pods are killed, grouped by label selector
</p></details>

---

What can deployments be used for?
<details><summary>Answer</summary><p>
ErmÃ¶glichen Deklarative Updates von Container Images in Pods.
</p></details>

---